데이터 분석

In [1]:
import json

data_path = '/content/drive/MyDrive/롯데정보통신ai/1_인텐트_분류.json'

with open(data_path,'r') as f:
    json_data = json.load(f)

data_type = []
for data in json_data:
    data_type.append(data['인텐트'])

# label 유형
print(set(data_type)) # {'결제_일반_질문', '결제_할인_질문', '구매_예약_요청', '배송_날짜_요청', '제품_시용_질문', '제품_정보_질문'}

for i in list(set(data_type)): # i[0] = 90, i[1] = 133, i[2] = 154, i[3] = 78, i[4] = 68, i[5] = 110
    print(f'{i}의 데이터 갯수 {data_type.count(i)}')

# data 개수
len(json_data) # 633


{'구매_예약_요청', '결제_일반_질문', '제품_시용_질문', '제품_정보_질문', '배송_날짜_요청', '결제_할인_질문'}
구매_예약_요청의 데이터 갯수 78
결제_일반_질문의 데이터 갯수 154
제품_시용_질문의 데이터 갯수 110
제품_정보_질문의 데이터 갯수 68
배송_날짜_요청의 데이터 갯수 133
결제_할인_질문의 데이터 갯수 90


633

In [2]:
data_type = []
for data in json_data:
    data_type.append(data['발화자'])

# label 유형
print(set(data_type)) # {'결제_일반_질문', '결제_할인_질문', '구매_예약_요청', '배송_날짜_요청', '제품_시용_질문', '제품_정보_질문'}

for i in list(set(data_type)): # i[0] = 90, i[1] = 133, i[2] = 154, i[3] = 78, i[4] = 68, i[5] = 110
    print(f'{i}의 데이터 갯수 {data_type.count(i)}')

# data 개수
len(json_data) # 633

{'s', 'c'}
s의 데이터 갯수 224
c의 데이터 갯수 409


633

중복 제거

In [3]:
import pandas as pd

json2pd = pd.DataFrame(json_data)
print(json2pd['발화문'].nunique())
print(json2pd.drop_duplicates(subset='발화문'))

633
    발화자                                                발화문       인텐트
0     c         실수로 스크류나사를 구입 못하였습니다 따로 구입할 수 있는 방법이 있을까요?  결제_일반_질문
1     c                                    어디서 구입할 수 있을까요?  결제_일반_질문
2     s                             방문 수령 불가능합니다. 양해 바랍니다.  결제_일반_질문
3     c                                 이런 상황인데도 구입 가능한가요?  결제_일반_질문
4     s                                     혼란을 드려서 죄송합니다.  결제_일반_질문
..   ..                                                ...       ...
628   c               포터탑차에 설치할 건데 추가로 배선 연장 얼마나 주문하면 될까요?  제품_정보_질문
629   c                         윈도우132 오버워치 옵션은 어떻게 돌아가나요?  제품_정보_질문
630   c                                      한국정발로 오는 건가요?  제품_정보_질문
631   c  화장실에 놀거구요한 평 정도 화장실에 놀거구요 화장실 이 외벽이라 겨울에 벽에서리가...  제품_정보_질문
632   c                                     삼성S10 은 5핀이에요?  제품_정보_질문

[633 rows x 3 columns]


bert를 이용한 classification model

In [4]:
!pip install transformers


feature가 2개니까 multi-task-learning으로 가자잇
1. 소 의도분류
2. 대 의도분류
3. user 구분

json-> tokenizer하기 위한 전처리

In [5]:
import random

total_data = []

# train # 9
# test # 1

for data in json_data:
    temp = []
    temp.append([data['발화자']][0])
    # if data['발화자'] == 'c':
        # temp.append(10)
    # if data['발화자'] == 's':
        # temp.append(11)
    temp.append([data['발화문']][0])
    if data['인텐트'] == '결제_일반_질문':
        temp.append(0)
    if data['인텐트'] == '결제_할인_질문':
        temp.append(1)
    if data['인텐트'] == '구매_예약_요청':
        temp.append(2)
    if data['인텐트'] == '배송_날짜_요청':
        temp.append(3)
    if data['인텐트'] == '제품_시용_질문':
        temp.append(4)
    if data['인텐트'] == '제품_정보_질문':
        temp.append(5)
    total_data.append(temp)

In [6]:
len(total_data)
print(total_data)

[['c', '실수로 스크류나사를 구입 못하였습니다 따로 구입할 수 있는 방법이 있을까요?', 0], ['c', '어디서 구입할 수 있을까요?', 0], ['s', '방문 수령 불가능합니다. 양해 바랍니다.', 0], ['c', '이런 상황인데도 구입 가능한가요?', 0], ['s', '혼란을 드려서 죄송합니다.', 0], ['s', '자유 모형톡은 아크릴 제작 부분 때문에 최소 수량 30개부터 제작이 가능합니다.', 0], ['s', '문의 주 신제품은 옵션 아래 추가 구성에서 색상 선택이 가능합니다.', 0], ['c', '리필 키트도 최대 1개씩만 구입 가능한가요?', 0], ['c', '482W /FW 모델 보상교환 482W 나 482FW 를 고려하고 있는데, 기존 제품을 반납해야 하는지요?', 0], ['c', '추가 상품 14번 핸들 그립 미니삼각 대구매 안 되나요?', 0], ['s', '문의 해주신 내용 안내드릴 수 있도록 하겠습니다. 주문 주 신상품은 A328 W R328 W로 화이트 냉장 냉동제품이 맞습니다.', 0], ['s', '문의하신 성능 모델은 판매 예정이 없는 모델임을 알려드립니다.', 0], ['c', '구매하려고 하는데 추가 상품 드러가서 별도 구매 1개월 단위로 가치 구매하면 되나요?', 0], ['s', '상향식 모델에는 SG15 가 기재되어 있습니다. 기술서 및 상품명을 확인 후 구매해 주시면 감사하겠습니다.', 0], ['s', '기존 가맹점으로 구매는 가능하지만  다른 대리점에 등록된 상태라면 기존 가맹점으로 구매할 경우 티아이디를 새로 부여받아야 합니다.', 0], ['c', '추가 옵션 문의 추가 옵션에 리모콘 외에 스마트폰 거치대 추가는 없는데 스카트폰 거치대는 어떻게 구매해야 하나요?', 0], ['s', '어떤 타입의 잉크를 찾는지 알려주시면 구매할 수 있는 주소 안내해드리겠습니다.', 0], ['c', '갤10/5G 는 지금 구매 안 되는 건가요?', 0], ['s', '추가해놨습니다. 구매하실 수 있습니다.

In [7]:
from sklearn.model_selection import train_test_split
train_data, test_data = train_test_split(total_data, test_size = 0.1, shuffle = True, random_state = 42)

In [52]:
print(train_data)
print(test_data)

[['s', '네, 삼성건조기 설치하러 11월 1일에 방문할 예정입니다.', 2], ['c', '다음 달에 국내에 출시될 아이폰 사전 예약이 지금 될까요?', 2], ['s', '말씀해 주신 대로 주문이 안 되신다면 수량을 2개로 지정하시고 배송 메세지에 OOO OOO 색으로 주세요.라고 남겨주세요.', 0], ['c', '제가 23일 날 어딜 가서 22일까지 꼭 받아야 하는데 빨리 보내주실 수 있을까요?', 3], ['s', '네, 현재 음향을 확인해 보십시오.', 4], ['c', '죄송합니다만, 정수기 물은 따로 시음이 어렵습니다.', 4], ['c', '테스트용 데스크탑 이거보다 더 사양 좋은거도 사용해봐도 될까요?', 4], ['c', 'K5 .2012년은 100번 주문하면 될까요?', 0], ['s', '배송요청 사항에 기재해 드렸습니다. 내일 금요일 우체국 택배로 도착 예정됩니다.', 0], ['s', '네, 아이폰 이번에 신규로 나온 모델로 사전 예약이 됩니다.', 2], ['s', '네 반품 회수로 진행되시며 그대로 포장하셔서 전달만 해주시면 되셔요.', 1], ['c', '네, 음향을 확인하셔도 됩니다.', 4], ['c', '정수기 냉장고 신기해서 그런데 한 번 만져봐도 될까요?', 4], ['s', '7L   14L 바스켓은 호환되어 사용 가능합니다.', 0], ['c', '빠른 배송 가능할까요?', 3], ['c', '다음 주에 출시하는 로봇청소기 예약 된다면 하고 싶어요.', 2], ['s', '쿠폰은 재구매 시 자동으로 발행됩니다.', 1], ['s', '추가 구성의 소프트웨어 항목에서 윈도우 추가 구매 가능하십니다 구매에 참고 부탁드리겠습니다.', 0], ['c', '냉장고 구매하면 원하는 날짜에 배송받을 수 있는거 맞죠?', 3], ['s', '배송 지정 기능은 현재는 종료되었습니다. 물류센터 배송제품이다보니다일 배송은 어려운 점 양해 바랍니다.', 3], ['c', '씨티, 현대, 롯데카드는 매일 번갈아가며 할인 혜택 적용되는데

tokenizer

In [9]:
!pip install konlpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 59.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 488.6/488.6 kB 44.6 MB/s eta 0:00:00


In [10]:
from konlpy.tag import Okt
okt = Okt()
print(okt.morphs('네, 삼성건조기 설치하러 11월 1일에 방문할 예정입니다.',stem = True))
a = okt.morphs('네, 삼성건조기 설치하러 11월 1일에 방문할 예정입니다.',stem = True)

['네', ',', '삼성', '건조기', '설치', '하다', '11월', '1일', '에', '방문', '하다', '예정', '이다', '.']


In [11]:
# model = BertForSequenceClassification.from_pretrained('klue/bert-base', num_labels=2)

In [12]:
from transformers import BertTokenizer, BertForSequenceClassification

model_name = 'klue/bert-base'
tokenizer = BertTokenizer.from_pretrained(model_name)
print(tokenizer('네, 삼성건조기 설치하러 11월 1일에 방문할 예정입니다.', truncation=True, padding =True))
print(tokenizer.tokenize('네, 삼성건조기 설치하러 11월 1일에 방문할 예정입니다.', truncation=True, padding =True))

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/289 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/248k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/495k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/425 [00:00<?, ?B/s]

Keyword arguments {'truncation': True, 'padding': True} not recognized.


{'input_ids': [2, 752, 16, 3840, 28193, 2015, 4198, 2205, 2030, 3718, 2429, 21, 2210, 2170, 4110, 2085, 3834, 12190, 18, 3], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}
['네', ',', '삼성', '##건조', '##기', '설치', '##하', '##러', '11', '##월', '1', '##일', '##에', '방문', '##할', '예정', '##입니다', '.']


In [13]:
# train_tokenize = []
# for i,j,k in train_data:
#     train_tokenize.append(tokenizer(j,truncation = True, padding = True))
#     # print(i,j,k)

# print(train_tokenize)

In [74]:
def convert_data2feature(datas, max_length, tokenizer):
    input_ids_features, attention_mask_features, token_type_ids_features, label_id_features = [], [], [], []

    count = 0
    for use,utt,lab in datas:
        count+= 1
        print(utt)
        # klue tokenize로 치환
        input_ids = tokenizer(utt)['input_ids'][:max_length]
        # print(input_ids)
        # print(input_ids[-1])
        # print([tokenizer.sep_token])
        input_ids[-1] =  3
        token_type_ids = tokenizer(utt)['token_type_ids'][:max_length]
        attention_masks = tokenizer(utt)['attention_mask'][:max_length]

        # padding 생성

        padding = [0] * (max_length - len(input_ids))
        input_ids += padding
        token_type_ids += padding
        attention_masks += padding
        assert max_length == len(input_ids) == len(token_type_ids) == len(attention_masks)
        label_id = lab

        # 변환한 데이터를 각 리스트에 저장
        input_ids_features.append(input_ids)
        attention_mask_features.append(token_type_ids)
        token_type_ids_features.append(attention_masks)
        label_id_features.append(label_id)
        # print(input_ids_features)

    # 변환한 데이터를 Tensor 객체에 담아 반환
    # print(input_ids_features)
    input_ids_features = torch.tensor(input_ids_features, dtype=torch.long)
    attention_mask_features = torch.tensor(attention_mask_features, dtype=torch.long)
    token_type_ids_features = torch.tensor(token_type_ids_features, dtype=torch.long)
    label_id_features = torch.tensor(label_id_features, dtype=torch.long)

    return input_ids_features, attention_mask_features, token_type_ids_features, label_id_features

In [75]:
import torch


convert_data2feature(train_data, 64, tokenizer)
print(len(train_data))

네, 삼성건조기 설치하러 11월 1일에 방문할 예정입니다.
[2, 752, 16, 3840, 28193, 2015, 4198, 2205, 2030, 3718, 2429, 21, 2210, 2170, 4110, 2085, 3834, 12190, 18, 3]
3
다음 달에 국내에 출시될 아이폰 사전 예약이 지금 될까요?
[2, 3729, 814, 2170, 3739, 2170, 4542, 2651, 7765, 4858, 6297, 2052, 3660, 14765, 35, 3]
3
말씀해 주신 대로 주문이 안 되신다면 수량을 2개로 지정하시고 배송 메세지에 OOO OOO 색으로 주세요.라고 남겨주세요.
[2, 4241, 2097, 1564, 2250, 4003, 4867, 2052, 1378, 859, 2250, 5209, 12395, 2069, 22, 2019, 2200, 4753, 2205, 24563, 9488, 29244, 2170, 27326, 2185, 27326, 2185, 1245, 6233, 27616, 2182, 18, 3609, 6962, 2223, 5971, 18, 3]
3
제가 23일 날 어딜 가서 22일까지 꼭 받아야 하는데 빨리 보내주실 수 있을까요?
[2, 1545, 2116, 4136, 2210, 721, 15901, 543, 2112, 4107, 2210, 2299, 2118, 677, 21372, 2275, 1889, 13964, 4828, 4209, 2223, 2477, 1295, 1513, 16809, 2182, 35, 3]
3
네, 현재 음향을 확인해 보십시오.
[2, 752, 16, 3738, 14047, 2069, 3915, 2097, 1160, 14924, 18, 3]
3
죄송합니다만, 정수기 물은 따로 시음이 어렵습니다.
[2, 7822, 11800, 2154, 16, 16111, 1093, 2073, 5542, 30231, 2052, 4258, 2219, 3606, 18, 3]
3
테스트용 데스크탑 이거보다 더 사양 

In [16]:
import torch.nn as nn
from transformers import BertPreTrainedModel, BertModel

class IntentClassifier(BertPreTrainedModel):

    def __init__(self, config):
        super(IntentClassifier, self).__init__(config)

        # BERT 모델
        self.bert = BertModel(config)

        # 히든 사이즈
        self.hidden_size = config.hidden_size

        # 분류할 라벨의 개수
        self.num_labels = config.num_labels


        # self.linear = nn.Linear(in_features=self.hidden_size, out_features=self.num_labels)
        self.fc1 = nn.Linear(in_features=self.hidden_size, out_features = 32)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(in_features=32, out_features=self.num_labels)
        self.softmax = nn.Softmax(dim=1)

    def forward(self, input_ids, attention_mask, token_type_ids):
        # BERT 출력에서 CLS에 대응하는 벡터 표현 추출
        # 선형 함수를 사용하여 예측 확률 분포로 변환

        outputs = self.bert(input_ids=input_ids, attention_mask = attention_mask, token_type_ids = token_type_ids)

        # (batch_size, max_length, hidden_size)
        bert_output = outputs[0] # CLS의 output

        # (batch_size, hidden_size)
        cls_vector = bert_output[:,0,:]

        # class_output : (batch_size, num_labels)
        cls_output = self.fc1(cls_vector)
        cls_output = self.relu(cls_output)

        cls_output = self.fc2(cls_output)
        # cls_output = self.softmax(cls_output)

        return cls_output

In [17]:
from torch.utils.data import (DataLoader, TensorDataset, RandomSampler)
from transformers import BertConfig
import torch.optim as optim
import torch
import numpy as np

def train(config):
    train_input_ids_features, train_attention_mask_features, train_token_type_ids_features, train_label_id_features = convert_data2feature(train_data, 64, tokenizer)
    train_features = TensorDataset(train_input_ids_features, train_attention_mask_features, train_token_type_ids_features, train_label_id_features)
    train_dataloader = DataLoader(train_features, sampler=RandomSampler(train_features), batch_size = config['batch_size'])

    # test_input_ids_features, test_attention_mask_features, test_token_type_ids_features, test_label_id_features = convert_data2feature(test_data, 64, tokenizer)
    # test_features = TensorDataset(test_input_ids_features, test_attention_mask_features, test_token_type_ids_features, test_label_id_features)
    # test_dataloader = DataLoader(test_features, sampler=RandomSampler(test_features), batch_size = config['batch_size'])

    bert_config = BertConfig.from_pretrained(pretrained_model_name_or_path='klue/bert-base')
    setattr(bert_config, "num_labels", config["num_labels"])

    # model = IntentClassifier.from_pretrained(pretrained_model_name_or_path='klue/bert-base', config=bert_config).cuda()
    model = IntentClassifier.from_pretrained(pretrained_model_name_or_path='klue/bert-base', config=bert_config)
    # loss를 계산하기 위한 함수
    loss_func = nn.CrossEntropyLoss()

    # 모델 학습을 위한 optimizer
    optimizer = optim.Adam(model.parameters(), lr=2e-5)

    # 모델의 정확도를 저장하기 위한 변수
    max_accuracy = 0
    global_step = 0
    for epoch in range(config["epoch"]):
        model.train()

        total_loss = []
        for step, batch in enumerate(train_dataloader):
            # batch = tuple(t.cuda() for t in batch)
            batch = tuple(t for t in batch)
            input_ids, attention_mask, token_type_ids, label_id = batch

            # 역전파 단계를 실행하기 전에 변화도를 0으로 변경
            optimizer.zero_grad()

            # 모델 예측 결과
            hypothesis = model(input_ids, attention_mask, token_type_ids)

            # loss 계산
            loss = loss_func(hypothesis, label_id)

            total_loss.append(loss.data.item())
            # loss 값으로부터 모델 내부 각 매개변수에 대하여 gradient 계산
            loss.backward()
            # 모델 내부 각 매개변수 가중치 갱신
            optimizer.step()

            # batch 당 저장할게
            print("Average loss : {}".format(np.mean(total_loss)))

        # epoch 당 저장할게
        bert_config.save_pretrained(save_directory=config["output_dir_path"])
        model.save_pretrained(save_directory=config["output_dir_path"])

In [81]:
def test(config):

    from transformers import BertTokenizer, BertForSequenceClassification

    model_name = 'klue/bert-base'
    tokenizer = BertTokenizer.from_pretrained(model_name)

    test_input_ids_features, test_attention_mask_features, test_token_type_ids_features, test_label_id_features = convert_data2feature(test_data, 64, tokenizer)
    test_features = TensorDataset(test_input_ids_features, test_attention_mask_features, test_token_type_ids_features, test_label_id_features)
    test_dataloader = DataLoader(test_features, sampler=RandomSampler(test_features), batch_size = config['batch_size'])

    # train_input_ids_features, train_attention_mask_features, train_token_type_ids_features, train_label_id_features = convert_data2feature(train_data, 64, tokenizer)
    # train_features = TensorDataset(train_input_ids_features, train_attention_mask_features, train_token_type_ids_features, train_label_id_features)
    # train_dataloader = DataLoader(train_features, sampler=RandomSampler(train_features), batch_size = config['batch_size'])



    bert_config = BertConfig.from_pretrained(pretrained_model_name_or_path=config['output_dir_path'])
    model = IntentClassifier.from_pretrained(pretrained_model_name_or_path='/content/drive/MyDrive/롯데정보통신ai/model.safetensors', config=bert_config).cuda()

    model.eval()

    total_hypothesis, total_labels = [], []
    for idx, batch in enumerate(test_dataloader):
        batch = tuple(t.cuda() for t in batch)
        input_ids, attention_mask, token_type_ids, label_id = batch

        with torch.no_grad():
            # 모델 예측 결과
            # hypothesis : (batch_size, num_labels)
            hypothesis = model(input_ids, attention_mask, token_type_ids)
            # 모델의 출력값에 softmax와 argmax 함수를 적용
            hypothesis = torch.argmax(hypothesis, dim=-1)

        # Tensor를 리스트로 변경
        hypothesis = hypothesis.cpu().detach().numpy().tolist()
        label_id = label_id.cpu().detach().numpy().tolist()

        total_hypothesis.extend(hypothesis)
        total_labels.extend(label_id)

        for index in range(len(input_ids)):
            input_tokens = tokenizer.convert_ids_to_tokens(input_ids[index])
            print(input_tokens)
            input_sequence = tokenizer.convert_tokens_to_string(input_tokens[1:input_tokens.index(tokenizer.sep_token)])
            print(input_tokens)
            print(input_sequence)

            predict = [hypothesis[index]]
            correct = [label_id[index]]

            # if hypothesis[index] == 0:
                # predict = '결제_일반_질문'

            print("입력 : {}".format(input_sequence))
            print("출력 : {}, 정답 : {}\n".format(predict, correct))

        from sklearn.metrics import accuracy_score
        print("Accuracy= {0:f}\n".format(accuracy_score(total_hypothesis, total_labels)))

In [76]:
config = {"mode": "train",
            # "train_data_path": os.path.join(root_dir, "train_datas_wordpiece.txt"),
            # "test_data_path": os.path.join(root_dir, "test_datas_wordpiece.txt"),
            "output_dir_path":'/content/drive/MyDrive/롯데정보통신ai',
            # "cache_dir_path": cache_dir,
            # "pretrained_model_name_or_path": "monologg/kobert",
            # "label_vocab_data_path": os.path.join(root_dir, "label_vocab.txt"),
            "num_labels": 6,
            "max_length": 64,
            "epoch":1,
            "batch_size":64,
            }

if (config["mode"] == "train"):
    train(config)
else:
    test(config)

네, 삼성건조기 설치하러 11월 1일에 방문할 예정입니다.
[2, 752, 16, 3840, 28193, 2015, 4198, 2205, 2030, 3718, 2429, 21, 2210, 2170, 4110, 2085, 3834, 12190, 18, 3]
3
다음 달에 국내에 출시될 아이폰 사전 예약이 지금 될까요?
[2, 3729, 814, 2170, 3739, 2170, 4542, 2651, 7765, 4858, 6297, 2052, 3660, 14765, 35, 3]
3
말씀해 주신 대로 주문이 안 되신다면 수량을 2개로 지정하시고 배송 메세지에 OOO OOO 색으로 주세요.라고 남겨주세요.
[2, 4241, 2097, 1564, 2250, 4003, 4867, 2052, 1378, 859, 2250, 5209, 12395, 2069, 22, 2019, 2200, 4753, 2205, 24563, 9488, 29244, 2170, 27326, 2185, 27326, 2185, 1245, 6233, 27616, 2182, 18, 3609, 6962, 2223, 5971, 18, 3]
3
제가 23일 날 어딜 가서 22일까지 꼭 받아야 하는데 빨리 보내주실 수 있을까요?
[2, 1545, 2116, 4136, 2210, 721, 15901, 543, 2112, 4107, 2210, 2299, 2118, 677, 21372, 2275, 1889, 13964, 4828, 4209, 2223, 2477, 1295, 1513, 16809, 2182, 35, 3]
3
네, 현재 음향을 확인해 보십시오.
[2, 752, 16, 3738, 14047, 2069, 3915, 2097, 1160, 14924, 18, 3]
3
죄송합니다만, 정수기 물은 따로 시음이 어렵습니다.
[2, 7822, 11800, 2154, 16, 16111, 1093, 2073, 5542, 30231, 2052, 4258, 2219, 3606, 18, 3]
3
테스트용 데스크탑 이거보다 더 사양 

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model.safetensors:   0%|          | 0.00/445M [00:00<?, ?B/s]

Some weights of IntentClassifier were not initialized from the model checkpoint at klue/bert-base and are newly initialized: ['fc2.bias', 'fc2.weight', 'fc1.weight', 'fc1.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Average loss : 1.8105769157409668
Average loss : 1.7955960631370544


KeyboardInterrupt: ignored

In [78]:
config = {"mode": "test",
            # "train_data_path": os.path.join(root_dir, "train_datas_wordpiece.txt"),
            # "test_data_path": os.path.join(root_dir, "test_datas_wordpiece.txt"),
            "output_dir_path":'/content/drive/MyDrive/롯데정보통신ai',
            # "cache_dir_path": cache_dir,
            # "pretrained_model_name_or_path": "monologg/kobert",
            # "label_vocab_data_path": os.path.join(root_dir, "label_vocab.txt"),
            "num_labels": 6,
            "max_length": 64,
            "epoch":1,
            "batch_size":64,
            }

if (config["mode"] == "train"):
    train(config)
else:
    test(config)

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
3
상품권은 설치 4주 후 발급됩니다.
[2, 9361, 2073, 4198, 24, 2223, 1943, 8224, 3598, 3606, 18, 3]
3
대량 구매 가능한가요?
[2, 7679, 4625, 3662, 2470, 18119, 35, 3]
3
3개 최고급형 18900원입니다. 기본형은 6500원입니다.
[2, 23, 2019, 3841, 2478, 2444, 28172, 2082, 2252, 12190, 18, 4116, 2444, 2073, 21629, 2252, 12190, 18, 3]
3
주문하신 상품은 16일로 지정하여 택배 발송 예정입니다. 택배사 사정에 따라 16일에 맞춰 가시거나 2 - 3일 지연되 실 수 있는 점 양해 부탁드립니다.
[2, 4867, 2205, 2250, 3959, 2073, 3879, 2210, 2200, 4753, 7488, 9678, 11786, 3834, 12190, 18, 9678, 2063, 5183, 2170, 3653, 3879, 2210, 2170, 5167, 6571, 9253, 22, 17, 23, 2210, 7035, 2496, 1329, 1295, 1513, 2259, 1540, 10797, 5527, 2343, 7316, 18, 3]
3
고객님께서 문의주신 빈센트 S8 브라운 색상 주문 가능합니다. 주문해 주시면 확인 하에 보내드리겠습니다.
[2, 3916, 2098, 2678, 2112, 7421, 2223, 2250, 27595, 55, 2196, 10424, 8817, 4867, 3662, 11800, 18, 4867, 2097, 12000, 2460, 3915, 1889, 2170, 4209, 6506, 2918, 2219, 3606, 18, 3]
3
사제 데크는 기아 신형 암잭으로 되있는데요. 기아 신형 숫잭과 현대 신형 암잭 같이 사야 되나요?
[2, 10705, 23690, 2259, 6194, 15920, 1383

In [82]:
config = {"mode": "test",
            # "train_data_path": os.path.join(root_dir, "train_datas_wordpiece.txt"),
            # "test_data_path": os.path.join(root_dir, "test_datas_wordpiece.txt"),
            "output_dir_path":'/content/drive/MyDrive/롯데정보통신ai',
            # "cache_dir_path": cache_dir,
            # "pretrained_model_name_or_path": "monologg/kobert",
            # "label_vocab_data_path": os.path.join(root_dir, "label_vocab.txt"),
            "num_labels": 6,
            "max_length": 64,
            "epoch":1,
            "batch_size":64,
            }

if (config["mode"] == "train"):
    train(config)
else:
    test(config)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


당일 9시 이전 결제 건에 한하여 당일 출고로 진행되며 그 이후 결제건은 익일 출고로 진행됩니다.
[2, 7494, 29, 2067, 4176, 6286, 570, 2170, 25268, 2173, 7494, 15139, 2200, 3750, 2496, 2307, 636, 3719, 6286, 2332, 2073, 1505, 2210, 15139, 2200, 3750, 3598, 3606, 18, 3]
3
네, 신규로 출시된 아이폰 모델로 사전 예약을 하실 수 있어요.
[2, 752, 16, 5089, 2200, 4542, 2897, 7765, 4347, 2200, 4858, 6297, 2069, 1889, 2477, 1295, 1513, 10283, 18, 3]
3
네, 이번주 토요일에 노트북 배송 가능합니다.
[2, 752, 16, 3686, 2223, 8414, 2170, 11161, 9488, 3662, 11800, 18, 3]
3
문의주신 건으로 신일 확인해 보았으나 충전코드의 경우 거치대에 부착되어 있는 상품으로 거치대만 따로 판매가 어렵다고 하셨습니다.
[2, 7421, 2223, 2250, 570, 6233, 28036, 3915, 2097, 1160, 2886, 4381, 7481, 14393, 2079, 3640, 6880, 2104, 2170, 9923, 2496, 2051, 1513, 2259, 3959, 6233, 6880, 2104, 2154, 5542, 3899, 2116, 4258, 4683, 1889, 3011, 2219, 3606, 18, 3]
3
안마의자 앉아서 테스트해볼 수 있는 거죠?
[2, 17824, 7672, 1379, 2227, 2112, 7453, 2097, 2345, 1295, 1513, 2259, 568, 2321, 35, 3]
3
네, 중량 확인해보세요.
[2, 752, 16, 16225, 3915, 2097, 2178, 5971, 18, 3]
3
현재 본 제품 구매시장 바구니 쿠폰은 없으며 MD 쿠폰과 레노